In [44]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string
import stopwords
from nltk.corpus import stopwords

In [45]:
tags = pd.read_csv("../data/movielens/tags.csv")

# EDA

In [46]:
tags["tag"] = tags["tag"].astype(str)
tags["char_length"] = tags["tag"].map(lambda x: len(x))
tags["word_length"] = tags["tag"].map(lambda x: len(x.split(" ")))
tags.head()

,userId,movieId,tag,timestamp,char_length,word_length
0,10,260,good vs evil,1430666558,12,3
1,10,260,Harrison Ford,1430666505,13,2
2,10,260,sci-fi,1430666538,6,1
3,14,1221,Al Pacino,1311600756,9,2
4,14,1221,mafia,1311600746,5,1


In [47]:
pd.set_option("display.float_format", "{:.2f}".format)
tags[["char_length", "word_length"]].describe()

,char_length,word_length
count,2328315.00,2328315.00
mean,11.09,1.66
std,5.69,0.92
min,1.00,1.00
25%,7.00,1.00
50%,10.00,1.00
75%,14.00,2.00
max,241.00,42.00


In [48]:
print(f"There are {tags["tag"].nunique()} unique words in tags.")

There are 153950 unique words in tags.


# Feature Engineering

In [49]:
# Special thanks to https://www.kaggle.com/tanulsingh077 for this function
def clean_text(text):
    """Make text lowercase, remove text in square brackets, remove links, remove punctuation
    and remove words containing numbers."""
    text = str(text).lower()
    text = re.sub(r"\[.*?\]", "", text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\w*\d\w*", "", text)
    return text

In [50]:
tags["tag"] = tags["tag"].apply(clean_text)
print(f"There are {tags["tag"].nunique()} unique words in tags.")

There are 141025 unique words in tags.


In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words("english")
more_stopwords = []  # add internet slang, etc.
# stop_words = stop_words + more_stopwords

def remove_stopwords(text):
    text = " ".join(word for word in text.split(" ") if word not in stop_words)
    return text

tags["tag"] = tags["tag"].apply(remove_stopwords)
print(f"There are {tags['tag'].nunique()} unique words in tags.")

OSError: No such file or directory: '/home/albot/nltk_data/corpora/stopwords/english'